In [33]:
import pandas as pd
import numpy as np

In [34]:
b = 15
f = 4
value_num = b + f

In [44]:
df = pd.read_csv('data/clean_dataframe')
df.set_index('Unnamed: 0', inplace=True)
df.index.names = ['date']
df.head()

,MSFT,DIS,HAS,JNJ,ADBE,V
date,,,,,,
2008-10-01,19.923723,26.028379,23.815720,46.272808,38.119999,14.006698
2008-10-02,19.750673,25.417543,22.762486,46.272808,35.220001,13.156832
2008-10-03,19.803337,25.061228,22.755554,45.160210,33.689999,12.985491
2008-10-06,18.742439,23.975292,21.979475,44.027092,30.709999,12.651946
2008-10-07,17.478403,22.541527,21.141045,42.655083,28.690001,11.685575


## Window Batching

In [36]:
batches_by_equity = []

for equity in df.columns:
    batches = []
    for i in range(len(df) - value_num):
        batches.append(df[equity][(0+i):(value_num+i)])
    np.random.shuffle(batches)
    batches_by_equity.append(batches)

###### Batch dataframe structure: `[equity_name][shuffled_interval_time_sequence_dataframe]`

In [37]:
# all_the_batches\[msft\]\[batch_number_243\]
batches_by_equity[0][243]

date
2019-10-17    137.123764
2019-10-18    134.885635
2019-10-21    135.886902
2019-10-22    133.864761
2019-10-23    134.718796
2019-10-24    137.369171
2019-10-25    138.144653
2019-10-28    141.541107
2019-10-29    140.206070
2019-10-30    141.953400
2019-10-31    140.736130
2019-11-01    141.079742
2019-11-04    141.894485
2019-11-05    141.806152
2019-11-06    141.413467
2019-11-07    141.609802
2019-11-08    143.278595
2019-11-11    143.425797
2019-11-12    144.368195
Name: MSFT, dtype: float64

In [38]:
proccessed_batches_by_equity = []
for equity_batch_set in batches_by_equity:
    processed_batch_set = []
    for batch in equity_batch_set:     
        full_seq = batch.pct_change().dropna(inplace=False)
        df_future = full_seq[-f:]
        df_past = full_seq[:b]

        min_max = max(abs(np.min(df_past)), abs(np.max(df_past)))
        df_past = np.interp(df_past, (-min_max, min_max), (-1, 1))
        df_future = np.interp(df_future, (-min_max, min_max), (-1, 1)) # future note: mayb find diff bounds than b_seq min_max
        processed_batch_set.append([df_past, df_future])
    proccessed_batches_by_equity.append(processed_batch_set)

###### Proccessed batches structure: `[equity number][sample number][past or future]`

## Train test split

In [39]:
dataset = []
for equity in proccessed_batches_by_equity:
    train_dataset = equity[0: int(0.8 * len(equity))]
    test_dataset = equity[int(0.8 * len(equity)):]
    equity_dataset = [train_dataset, test_dataset]
    dataset.append(equity_dataset)

###### dataset structure: `[equity number][train or test][sample number][past or future]`

In [40]:
# MSFT, train dataset, 246th sample
dataset[0][0][245]

[array([ 0.28896818, -0.26771403,  0.47088437, -0.59816664,  0.19047819,
        -0.30928269,  0.19120271, -0.37379045,  1.        ,  0.32946162,
        -0.14152228,  0.26576044,  0.41681932, -0.09076465, -0.22452409]),
 array([-0.22452409, -0.42166197,  0.21055015, -0.01845843])]

In [41]:
dataset = np.array(dataset)

In [42]:
np.save('data/dataset', dataset)